<a href="https://colab.research.google.com/github/jinsusong/21-study-paper-review/blob/main/Transformer_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Transformer : Attention is All You Need

2021년 기준으로 최신 고성능 모델들은 Transformer 아키텍처를 기반으로 함 

GPT : Transformer의 디코더 아키텍처를 활용
BERT : Transformer의 인코더 아키텍처를 활용



마운트 구글 드라이브

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%pwd

""" 
Use this javascript code in inspect>console so you wont need to click the page every 15 min:

########################
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
########################

"""

change current path to where the working project folder is at 

In [ ]:
%cd drive/MyDrive/projects/transformers_translation/

### Step 0 : Get The Data 

upload the data to our current path and unzip it (numcomment and run this only once)

In [ ]:
# # data is from: https://www.statmt.org/europarl/ you can use this or just upload your own data
# %cd data
# !wget https://www.statmt.org/europarl/v7/de-en.tgz
# !tar -xvf de-en.tgz
# %cd ..
# %pwd

get non breaking prefixs

In [ ]:
# get non_breaking_prefixes from https://github.com/moses-smt/mosesdecoder/tree/master/scripts/share/nonbreaking_prefixes
# then rename them to: "nonbreaking_prefix.en" and "nonbreaking_prefix.de" and put them in your data folder so we dont consider the
# dot in 'mr.jackson' as the end of a sentence

### Step 1 : Importing Dependencies

In [ ]:
import numpy as np
import math 
import re
import time # to see how long it takes in training


In [ ]:
%tensorflow_version 2.x

import tensorflow as tf
from tensorflow.keras import layers 
import tensorflow_datasets as tfds # tools for the tokenizer 



### Step 2 : Data Preprocessing 

read files

In [ ]:
with open("data/europarl-v7.de-en.en", mode='r', encoding="utf-8") as f:
    text_en = f.read()

with open("data/europarl-v7.de-en.de", mode='r', encoding="utf-8") as f:
    text_de = f.read()

print(text_en[:50])
print(text_de[:50])



In [ ]:
with open("data/nonbreaking-prefix.en", mode='r', encoding="utf-8") as f: 
    non_breaking_prefix_en = f.read()

with open("data/nonbreaking-prefix.de", mode='r', encoding="utf-8") as f:
    non_breaking_prefix_de = f.read()

print(non_breaking_prefix_en[:5])
print(non_breaking_prefix_de[:5])


Cleaning

In [ ]:
# 해석 필요 
for prefix in non_breaking_prefix_en:
    text_en = text_en.replace(prefix, prefix + '###')

text_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".###", text_en)
text_en = re.sub(r"\.###",'',text_en)
text_en = re.sub(r" +", ' ', text_en)
text_en = text_en.replace('###',' ')

text_en = text_en.split("\n")

for prefix in non_breaking_prefix_de:
    text_de = text_de.replace(prefix, prefix + '###')
text_de = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".###", text_de)
text_de = re.sub(r"\.###",'',text_de)
text_de = re.sub(r" +",' ',text_de)
text_de = text_de.replace('###',' ')

text_de = text_de.split("\n")





### Tokenizing


In [ ]:
tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    text_en, target_vocab_size=8000
)

tokenizer_de = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    text_de, target_vocag_size=8000
)





In [ ]:
VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2
VOCAB_SIZE_DE = tokenizer_de.vocab_size + 2 

# we put start and tokens as size-1 and size-2 which are the same as 
# tokenizer_size and tokenizer_size +1 because the words are from [0 to ts -1]
# tokenizer_en.encode(sentence) give a list then list + list + list appends them

input = [[VOCAB_SIZE_EN-2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN-1]
         for sentence in text_en]

outputs = [[VOCAB_SIZE_DE-2] + tokenizer_de.encode(sentence) + [VOCAB_SIZE_DE-1]
          for sentence in text_de]




###Remove too long sentences

- Why? (1) because when we pad we will have a hugeeee ram issuie for example sentence sizes of 1,100,2 when we pad they become 100,100,100 which we would rather loose that 100 than pad all to 100 (2) takes too much time to train

In [ ]:
MAX_LENGTH = 20 # we will still have a lot of data with max len of 20 

# this part. why we do it is a bit tricky. pay attention why we do it like this:
idx_to_remove = [count for count, sent in enumerate(inputs)
if len(sent) > MAX_LENGTH]

# we remove in reversed because of shifting issuies when we satrt from begining
for idx in reversed(idx_to_remive):
    del inputs[idx]
    del outputs[idx]

# same stuff for outputs > 20 
idx_to_remove = [count for count, sent in enumerate(outputs)
if len(sent) > MAX_LENGTH]

for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]

 

### input / output creation

1. padding
2. batching

In [ ]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                       value=0,
                                                       padding='post',
                                                       maxlen = MAX_LENGTH)

outpus = tf.keras.preprocessing.sequence.pad_sequences(outputs,
                                                       value=0,
                                                       padding='post',
                                                       maxlen = MAX_LENGTH)



In [ ]:
BATCH_SIZE =64
BUFFER_SIZE = 20000 # how much data to keep

# now we turned our data into a dataset 
dataset = tf.data.Dataset.from_tensort_slices((inputs, outputs))

#this is something that improves the way the dataset is stored. it increases
# the speed of accessing the data which increases training speed in return :
data = dataset.cache()

#now we shuffle in batches
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

#this increases the speed even further:
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)


### Step 3 : Model Building

- A - Positional Encoding ( look at the formula in the paper)

In [ ]:
class PositionalEncoding(layers.Layer):
    
    def __init__(self):
        # 이 위치 인코더는 레이어의 하위 항목으로 만들어 레이어가 가지고 있는 모든 속성을 가집니다
        super(PositionalEncoding, self).__init__()

    def get_angles(self, pos, i, d_model):
        """
        :pos: (seq_len, 1) 문장 내 단어의 색인 [0 ~ 19]
        :i: 임베딩의 치수 (각각 200 치수) 그 다음> [0 ~ 199]
        :d_model: 내장형 크기(예: glove  크기 200)
        :return: (seq_len, d_model) 왜? 우리는 그 단어의 모든 위치 대 모든 차원의 인코딩을 얻고 있다.
        """
           # PE (pos,2i) = sin (pos / 10000^(2i / d_model) )
        # PE (pos,2i+1) = cos (pos / 10000^(2i / d_model) )
        angles = 1 / np.power(10000., (2*(i//2))/np.float32(d_model))
        return pos * angles # dim: (seq_len, d_model)

    def call(self, inputs):
        # input.shape = [batch_size, multihead_size(sz=8), each word (pos), that words embedding]
        # 우리는 그들의 위치를 고려하여 입력값을 변경하지 않고, 우리는 단지 입력으로부터 딤을 얻고 완전히 개별적으로 pos 인코딩을 계산하여 끝에 쌓을 뿐이다.
        seq_length = inputs.shape.as_list()[-2] # basically the pos
        d_model = inputs.shape.as_list()[-1] # basically the embedded values
        angles = self.get_angles(np.arange(seq_length)[:, np.newaxis],
                                 np.arange(d_model)[np.newaxis, :],
                                 d_model)
        angles[:, 0::2] = np.sin(angles[:, 0::2])
        angles[:, 1::2] = np.cos(angles[:, 1::2])
        # 우리는 이것을 할 수 있다. 왜냐하면 입력과 인코딩이 같은 치수를 가져야 하기 때문이다. 그래서 우리는 0을 넣지 않는 새로운 축을 만들기 위해서… 모든 배치에 대해 같은 조광을 복사하고...
        pos_encoding = angles[np.newaxis, ...]
        # 이제 우리는 입력과 그들의 pos_encodings를 둘 다 반환해야 하지만 우리는 np에 pos_codings가 있어서 그것들을 tf로 만든다
        return inputs + tf.cast(pos_encoding, tf.float32)



### B - Attention 
 - Attention computation ( see the formula in the paper)

In [ ]:
def scaled_dot_product_attention(queries, keys, values, mask):
    # Q*K는 [output_len, d_model] * [d_model, input_len]이며 영어와 프랑스어 모두 20입니다.
    # transpose_b = True는 키를 키 방향으로 돌리게 됩니다. T는 각각 이 dim : [batch_size, nb_proj, seq_len, d_proj]이므로 전치 시 [a,b,c,d] * []가 된다.
    product = tf.matmul(queries, keys, transpose_b = True)
    keys_dim = tf.cast(tf.shape(keys)[-1], tf.float32) # makes the dim_num float
    scaled_product = product / tf.math.sqrt(keys_dim) # scalse it (formula stuff)

    #왜냐하면 논문이 말한 이 마스크는 프로그램이 feauture를 보는 것을 막기 위해 선택적이기 때문이다. 
    #왜냐하면 우리가 역프로포즈를 할 때 그들은 그들 앞에 있는 것들을 고려할 것이기 때문에 
    #우리는 이것을 멈추기 위해 그들에게 -1e9를 더해서 소프트맥스 후에 그들에게 0이 된다.

    if mask is not None:
        scaled_product +=(mask * -1e9)

    #소프트맥스를 마지막 축을 따라 적용한다. 왜냐하면 우리는 소프트맥스의 합이 input_len에 1 scale_product = [output_len] -> softmax on input_len이 되기를 원하기 때문에 
    #기본적으로 out_len에 대한 프로브를 동일하게 유지하지만 out_len에 대한 프로브를 찾는다.
    probs = tf.nn.softmax(scaled_product, axis = -1)

    # attention = [output_len. input_len] * [input_len, d_model] = [output_len, d_model]
    # 그래서 이제 우리는 각각의 출력 단어에 대한 d_model 가중치를 가지고 있으며, 각각의 out_lens에 대한 예측을 보기 위해 앞으로 전달할 것이다.
    attention = tf.matmul(probs, value)

    return attention
    

In [ ]:
# import numpy as np
# 이것은 단지 당신이 dims에 대한 코드의 줄에서 어떤 일이 일어나는지 보기 위한 테스트이다 
#(4dimes에서 우리가 깨달은 것은 matmul이 다른 dims를 배치 크기와 다른 것으로 간주하기 때문에
# 마지막 두 개에 대해서만 멀티를 수행하였다). (tf.matmul(a, b, transpose_b= True)

# a = np.arange(24).reshape(1,2,3,4)
# a = tf.convert_to_tensor(a, np.float32)
# b = np.arange(24).reshape(1,2,3,4)
# b = tf.convert_to_tensor(b, np.float32)
# product = tf.matmul(a, b, transpose_b=True)
# print(product.shape)

### Multi-Head attention sublayer

In [ ]:
class MultiHeadAttention(layers.Layer):
    def __init__(self, nb_proj):
        """
        :nb_proj : the number of projections for the multihead
        """
        super(MultiHeadAttention, self).__init__()
        self.np_proj = nb_proj

    
    #이것은 init와 동일하지만 우리가 객체를 처음으로 사용할 때 발생한다, init에서는 객체를 만들 때 호출되었다.
    def build(self, input_shape):
        self.d_model = input_shape[-1]
        # 우리는 그것들이 분리될 수 있는지 확인하고 싶다.
        assert self.d_model % self.nb_proj == 0
        # 우리는 그것을 정수로 만들기 위해 2개의 슬래시를 사용한다.
        self.d_proj = self.d_model // self.nb_proj

        self.query_lin = layers.Dense(self.d_model)
        self.key_lin = layers.Dense(self.d_model)
        self.value_lin = layers.Dense(self.d_model)
        self.final_lin = layers.Dense(self.d_model)

    def split_proj(self, inputs, batch_size):
        """
        : inputs : [batch_size, seq_len(20), d_model(prev layer dim)]

        : return: 
            dims = [batch_size, nb_proj, seq_len, d_proj]
            nb_proj는 cnn의 채널과 같습니다. 기본적으로 d_model을 nb_proj * d_proj로 분할하여 d_model/nb_proj를 찾습니다.
        """        
        new_shape = (batch_size, -1, self.nb_proj, self.d_proj)
        #here we will get: [ batch_sz, seq_len, nb_proj, d_proj]

        splited_inputs = tf.reshape(inputs, shape=new_shape)

        # so we need to reshape it to : [batch_size, nb_proj, seq_len, d_proj]
        return tf.transpose(splited_inputs, perm=[0,2,1,3])

    def call(self, queries, keys, values, mask):
        batch_size = tf.shape(queries)[0]

        queries = self.query_lin(queries)
        keys = self.key_lin(keys)
        values = self.value_lin(vlaues)

        # 이제 우리는 프로즈를 만들기 위해 그것들을 나누었다.
        queries = self.split_proj(queries, batch_size)
        keys = self.split_proj(keys, batch_size)
        values = self.split_proj(values, batch_size)

        #each of the q, k , v are [batch_size, nb_proj, seq_len, d_proj]
        attention = scaled_dot_product_attention(queries, keys, vlaues, mask)

        # 이제 위에서 했던 갈라진 부분을 뒤집을 거야 : reshape + concat 
        attention = tf.greanspose(attention, perm=[0,2,1,3])
        # we have [batch_size, seq_len, nb_proj, d_proj] so now we concat 2,3 
        concat_attention = tf.reshapre(attention, shape=(batch_size, -1, self.d_model))
        outputs = self.final_lin(concat_attention)
        return outputs 


### C - Encoder 

In [ ]:
class EncoderLayer(layers.Layer):
    
    def __init__(self, FFN_units, nb_proj, dropout):
        """
        :FFN_units: 
            feed forward networks units : the number of units for the
            feed forward which you can see in the encoder part of the 
            paper ( right after the attention there is a feed forward...)
        : nb_project:
            the number of projections we have (8)

        : dropout : 
            the dropout rate e.g. 0.3 
        """

        super(EncoderLayer, self).__init__()
        self.FFN_units = FFN_units
        self.nb_proj = nb_proj
        self.dropout = dropout
    
    # 우리는 인코더를 만들 때 우리가 원하는 많은 바들이 없기 때문에 이것을 사용한다. 그래서 우리가 대신 '빌드' 함수를 사용할 때 우리는 그것들을 얻을 수 없다.
    def build(self, input_shape):
        self.d_model = input_shape[-1]
        # we first build the object for the multi-head-attention
        self.multi_head_attention = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate=self.dropout)
        self.norm_1 = layers.LayerNormalization(epsilon=1e-6)

        self.dense_1 = layers.Dense(units = self.FFN_units, activation="relu")
        self.dense_2 = layers.Dense(units= self.d_model, activation="relu")
        self.dropout_2 = layers.Dropout(rate=self.dropout)
        self.norm_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, mask, training):
        """
        : mask: which we will apply in the multi-head attention
        : training:
            모델이 오버피팅되지 않도록 교육하는 동안 드롭아웃을 사용합니다=참
            하지만 우리는 테스트만 할 때 그것을 사용하지 않습니다(일명 train=false).
        """
        # 아키텍처를 살펴보면 인코더의 모든 쿼리/키/밸이 이전 계층에서 얻은 입력과 동일한 어레이임을 알 수 있습니다.
        attention = self.multi_head_attention(inputs, inputs, inputs, mask)

        # dropout + normalization after the attention
        attention = self.dropout_1(attention, training = training)
        # we do + inputs here 이유는 아키텍처에서 그것들이 결과 주의에 대한 이전 입력에 여전히 일치하기 때문에 우리는 그것을 정규화한다.
        attention = self.norm_1(attention+inputs)

        # now we do the dense in our FFN:
        outputs = self.dense_1(attention)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_2(outputs)
        outputs = self.norm_2(outputs + attention)

        return outputs 


    


In [ ]:
class Encoder(layers.Layer):
    
    def __init__(self,
                 nb_encoding_layers,
                 FFN_units, 
                 nb_proj,
                 dropout,
                 vocab_size,
                 d_model,
                 name="encoder"):
        # we put name = name here because the name is something that belongs to the layers class. so we tell it to use name="encoder"
        super(Encoder, self).__init__(name=name)
        self.nb_encoding_layers = nb_encoding_layers # the number of encoders in a row 
        self.d_model = d_model # the size of the output e.g glove(200)

        # we give vocab size for it to know the maxium number used in vocab 
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout)
        self.enc_layers = [EncoderLayer(FFN_units, nb_proj, dropout)
            for _ in range(self.nb_encoding_layers)]
        
    def call(self, inputs, mask, training):
        # look at the paper's architecture while doing these embedding with maybe glove eights .. 
        outputs = self.embedding(inputs)
        # 우리가 이것을 한 이유는 3,4항의 종이에 쓰인 것 때문에 그들은 그들이 곱하기라고 말했다.
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        #this will give us the concat : outputs + pos_encoding
        outputs = self.pos_encoding(outputs)
        #이제 우리는 모든 인코딩 레이어 전에 드롭아웃을 한다.
        # we give it training=bool -> so dont do dropout when training=false
        outputs = self.dropout(outputs, training)

        # 지금은 EmbeddingLayer를 한 번이 아니라 여러 번 해요.
        for i in range(self.nb_encoding_layers):
            # 따라서 다음 매개 변수를 사용하여 각 (i)번째 인코더에 적용합니다.
            outputs = self.enc_layers[i](outpus, mask, training)

        return outputs





### D - Decoder 

In [ ]:
class DecoderLayer(layers.Layer):
    def __init__(self, FFN_units, nb_proj, dropout):
        super(DecoderLayer, self).__init__()
        self.FFN_units = FFN_units
        self.nb_proj = nb_proj
        self.dropout = dropout 

    def build(self, input_shape):
        self.d_model = input_shape[-1]

        # MHA 1
        self.multi_head_attention_1 = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate=self.dropout)
        self.norm_1 = layers.LayerNormalization(epsilon=1e-6)

        # MHA 2 
        self.multi_head_attention_2 = MultiHeadAttention(self.nb_proj)
        self.dropout_2 = layers.Dropout(rate=self.dropout)
        self.norm_2 = layers.LayerNormalization(epsilon=1e-6)

        #FFN
        self.dense_1 = layers.Dense(units = self.FFN_units, activation='relu')
        self.dense_2 = layers.Dense(units = self.d_model)
        self.dropout_3 = layers.Dropout(rate = self.dropout)
        self.norm_3 = layers.LayerNormalization(epsilon=1e-6)

    
    def call(self, inputs, enc_outputs, mask_1, mask_2, training):
        # check the architecture in the paper to see why we do these

        # this is the 1# attention 
        attention = self.multi_head_attention_1(inputs, inputs, inputs, mask_1)
        # we give it training=bool -> so dont do dropout when training=false 
        attention = self.dropout_1(attention, training)
        attention = self.norm_1(attention + inputs)

        # this is the 2# attention. this is ALOT different than before one pay attention
        attention_2 = self.multi_head_attention_2(attention,
                                                  enc_outputs,
                                                  enc_outputs,
                                                  mask_2)
        
        #we give it training=bool -> so dont do dropout when training=fasle
        attention_2 = self.dropout_2(attention_2, training)
        attention_2 = self.norm_2(attention_2 + inputs)

        # the denses 
        outputs = self.dense_1(attention_2)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_3(outputs, training)
        outputs = self.norm_3(outputs + attention_2)

        return outputs 

        



In [ ]:
class Decoder(layers.Layer):

    def __init__(self,
                 nb_decoding_layers,
                 FFN_units,
                 nb_proj,
                 dropout,
                 vocab_size,
                 d_model,
                 name="decoder"):
        super(Decoder, self).__init__(name=name)
        self.nb_decoding_layers = nb_decoding_layers # the number of encoders in a row 
        self.d_model = d_model # the size of the output e.g glove(200)

        # we give vocab size for it to know the maximum number used in vocab 
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout)

        self.dec_layers = [DecoderLayer(FFN_units, nb_proj, dropout)
            for _ in range(nb_decoding_layers)]

    def call(self, inputs, enc_outputs, mask_1, mask_2, training):
        # look at the paper's architecture while doing these embedding with maybe glove weights ... 
        outputs = self.embedding(inputs)
        # the reason why we did this was vecause of what was writtent on the paper in secssion 3.4  which they said they multiplied
        # it by squt of d_model 
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        #this will give us the concat : outputs + pos_encoding
        outputs = self.pos_encoding(outputs)
        #now we do dropout before all the encoding layers 
        # we give it training=bool -> so dont do dropout when training=false 
        outputs = self.dropout(outputs, training)

        # now we do the EmbeddingLayer a couple of times, not just once
        for i in range(self, nb_decoding_layers):
            # so we apply it to the (i)th encoder in each for with these params:
            outputs = self.dec_layers[i](outputs,
                                         enc_outputs,
                                         mask_1,
                                         mask_2, 
                                         training)
        return outputs 

        


